<h1>Data Aggregation and Group Operations</h1>

Categorizing a dataset and applying a function to each group, whether an aggregation or transformation, is often a critical component of a data analysis workflow.
Pandas provides a flexible <b>groupby</b> interface, enabling us to slice, dice, and summarize datasets in a natural way.

<h3>GroupBy Mechanics</h3>

In the first stage of the process, data contained in a pandas object, whether a Series, DataFrame, or otherwise, is split into groups based on one or more keys that we provide. The splitting is performed on a particular axis of an object. For example, a DataFrame can be grouped on its rows (axis = 0) or its columns (axis = 1). Once this is done, a function is applied to each group, producing a new value. Finally, the results of all those function applications are combined into a result object. The form of the resulting object will usually depend on what's being done to the data.

![alt Text](Images/DataAggregation/da_group_agg.png)

Each grouping key can take many forms, and the keys do not have to be all of the same type:
<ul>
    <li>A list or array of values that is the same length as the axis being grouped</li>
    <li>A value indicating a column name in a DataFrame</li>
    <li>A dict or Series giving a correspondence between the values on the axis being grouped and the group names</li>
    <li>A function to be invoked on the axis index or the individual labels in the index</li>
</ul>

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.DataFrame({
    'key1': ['a', 'a', 'b', 'b', 'a'],
    'key2': ['one', 'two', 'one', 'two', 'one'],
    'data1': np.random.randn(5),
    'data2': np.random.randn(5)
})

In [3]:
df

,key1,key2,data1,data2
0,a,one,-1.576350,0.460778
1,a,two,-0.541179,-0.873254
2,b,one,0.061983,-0.397402
3,b,two,-0.274277,0.645504
4,a,one,-1.427910,1.108724


Now, suppose we wanted to compute the mean of the data1 column  using the labels from key1. There are a number of ways to do this. One is to access data1 and call groupby with the column at key1:

In [4]:
grouped = df['data1'].groupby(df['key1'])

In [5]:
grouped

In [6]:
grouped.mean()

key1
a   -1.181813
b   -0.106147
Name: data1, dtype: float64

Here, the data has been aggregated according to the group key, producing a new Series that is now indexed by the unique values in the key1 column.
The result index has the name 'key1' because the DataFrame column <b>df['key1']</b> did.

If instead we had passed multiple arrays as a list, we'd get something different:

In [7]:
means = df['data1'].groupby([df['key1'], df['key2']]).mean()

In [8]:
means

key1  key2
a     one    -1.502130
      two    -0.541179
b     one     0.061983
      two    -0.274277
Name: data1, dtype: float64

Here, we grouped the data using two keys, and the resulting Series now has a hierarchial index consisting of the unique paris of keys observed:

In [9]:
means.unstack()

key2,one,two
key1,,
a,-1.502130,-0.541179
b,0.061983,-0.274277


In the example below, the group keys are all Series, though they can be any arrays of the right length:

In [10]:
states = np.array(['Ohio', 'California', 'California', 'Ohio', 'Ohio'])

In [11]:
years = np.array([2005,2005,2006,2005,2006])

In [12]:
df

,key1,key2,data1,data2
0,a,one,-1.576350,0.460778
1,a,two,-0.541179,-0.873254
2,b,one,0.061983,-0.397402
3,b,two,-0.274277,0.645504
4,a,one,-1.427910,1.108724


In [13]:
df['data1'].groupby([states, years]).mean()

California  2005   -0.541179
            2006    0.061983
Ohio        2005   -0.925314
            2006   -1.427910
Name: data1, dtype: float64

Frequently the grouping information is found in the same DataFrame as the data we want to work on. In that case, we can pass column names as the group keys:

In [14]:
df

,key1,key2,data1,data2
0,a,one,-1.576350,0.460778
1,a,two,-0.541179,-0.873254
2,b,one,0.061983,-0.397402
3,b,two,-0.274277,0.645504
4,a,one,-1.427910,1.108724


In [15]:
df.groupby(['key1']).mean()

,data1,data2
key1,,
a,-1.181813,0.232082
b,-0.106147,0.124051


In [16]:
df.groupby(['key1', 'key2']).mean()

data1     data2
key1 key2                    
a    one  -1.502130  0.784751
     two  -0.541179 -0.873254
b    one   0.061983 -0.397402
     two  -0.274277  0.645504

Here, we can notice that in the first case df.groupby('key1').mean() that there is no key2 column in the  result. Because df['key2'] i snot numeric data, it is said to be a nuisance column, which is therefore excluded from the result. By default, all of the numeric columns are aggregated, though it is possible to filter down to a subset, as we'll see soon.

Regardless of the objective in using <b>groupby</b>, a generally useful GroupBy method is size, which returns a Series containing group sizes:

In [17]:
df

,key1,key2,data1,data2
0,a,one,-1.576350,0.460778
1,a,two,-0.541179,-0.873254
2,b,one,0.061983,-0.397402
3,b,two,-0.274277,0.645504
4,a,one,-1.427910,1.108724


In [18]:
df.groupby(['key1', 'key2']).size()

key1  key2
a     one     2
      two     1
b     one     1
      two     1
dtype: int64

<b>Note: Any missing values in a group key will be excluded from the result</b>

<h3>Iterating Over Groups</h3>

The GroupBy object supports iteration, generating a sequence of 2-tuples containing the group name along with the chunk of data. Consider the following:

In [19]:
df

,key1,key2,data1,data2
0,a,one,-1.576350,0.460778
1,a,two,-0.541179,-0.873254
2,b,one,0.061983,-0.397402
3,b,two,-0.274277,0.645504
4,a,one,-1.427910,1.108724


In [20]:
for name, group in df.groupby('key1'):
    print(name)
    print(group)

a
  key1 key2     data1     data2
0    a  one -1.576350  0.460778
1    a  two -0.541179 -0.873254
4    a  one -1.427910  1.108724
b
  key1 key2     data1     data2
2    b  one  0.061983 -0.397402
3    b  two -0.274277  0.645504


In the case of multiple keys, the first element in the tuple will be a tuple of key values:

In [21]:
for (k1,k2), group in df.groupby(['key1', 'key2']):
    print((k1,k2))
    print(group)

('a', 'one')
  key1 key2    data1     data2
0    a  one -1.57635  0.460778
4    a  one -1.42791  1.108724
('a', 'two')
  key1 key2     data1     data2
1    a  two -0.541179 -0.873254
('b', 'one')
  key1 key2     data1     data2
2    b  one  0.061983 -0.397402
('b', 'two')
  key1 key2     data1     data2
3    b  two -0.274277  0.645504


Of course, we can choose to do whatever we want with the pieces of data. A recipe we may find useful is computing a dict of the data pieces as a one-liner:

In [22]:
pieces  = dict(list(df.groupby('key1')))

In [23]:
pieces

{'a':   key1 key2     data1     data2
 0    a  one -1.576350  0.460778
 1    a  two -0.541179 -0.873254
 4    a  one -1.427910  1.108724, 'b':   key1 key2     data1     data2
 2    b  one  0.061983 -0.397402
 3    b  two -0.274277  0.645504}

In [24]:
pieces['b']

,key1,key2,data1,data2
2,b,one,0.061983,-0.397402
3,b,two,-0.274277,0.645504


By default groupby groups on axis = 0, but we can group on any of the other axes.
For example, we could group the columns of our example df here by dtype like so:


In [25]:
df.dtypes

key1      object
key2      object
data1    float64
data2    float64
dtype: object

In [26]:
grouped = df.groupby(df.dtypes, axis = 1)

In [27]:
for dtype, group in grouped:
    print(dtype)
    print(group)

float64
      data1     data2
0 -1.576350  0.460778
1 -0.541179 -0.873254
2  0.061983 -0.397402
3 -0.274277  0.645504
4 -1.427910  1.108724
object
  key1 key2
0    a  one
1    a  two
2    b  one
3    b  two
4    a  one


<h3>Selecting a Column or Subset of Columns</h3>

Indexing a GroupBy object created from a DataFrame with a column name or array of column names has the effect of column subsetting for aggregation. This means that:

In [28]:
df

,key1,key2,data1,data2
0,a,one,-1.576350,0.460778
1,a,two,-0.541179,-0.873254
2,b,one,0.061983,-0.397402
3,b,two,-0.274277,0.645504
4,a,one,-1.427910,1.108724


<pre>
df.groupby('key1')['data1']
df.groupby('key1')[['data2']]
are syntactice sugar for:
df['data1'].groupby(df['key1'])
df[['data2']].groupby(df['key1'])
</pre>

Especially for large datasets, it may be desirable to aggregate only a few columns. For example, in the preceding dataset, to compute means for just the data2 column and get the result as a DataFrame, we could write:


In [29]:
df

,key1,key2,data1,data2
0,a,one,-1.576350,0.460778
1,a,two,-0.541179,-0.873254
2,b,one,0.061983,-0.397402
3,b,two,-0.274277,0.645504
4,a,one,-1.427910,1.108724


In [30]:
df.groupby(['key1', 'key2'])[['data2']].mean()

data2
key1 key2          
a    one   0.784751
     two  -0.873254
b    one  -0.397402
     two   0.645504

The object returned by this indexing operation is a grouped DataFrame if a list or array is passed or a grouped Series if only a single columns name is passed as a scalar:

In [31]:
s_grouped = df.groupby(['key1', 'key2'])['data2']

In [32]:
s_grouped

In [33]:
s_grouped.mean()

key1  key2
a     one     0.784751
      two    -0.873254
b     one    -0.397402
      two     0.645504
Name: data2, dtype: float64

<h3>Grouping with Dicts and Series</h3>

Grouping information may exists in a form other than an array. Let's consider another example DataFrame:

In [34]:
people= pd.DataFrame(np.random.randn(5,5),
                    columns = ['a', 'b', 'c', 'd', 'e'],
                    index = ['Joe', 'Steve', 'Wes', 'Jim', 'Travis'])

In [35]:
people

,a,b,c,d,e
Joe,0.193067,-0.690679,-0.134004,0.718496,-0.360156
Steve,-0.031958,-0.407491,0.164615,-0.294408,-0.572945
Wes,-1.234819,0.529197,0.670012,-1.667063,-0.546139
Jim,0.918262,-2.794351,-1.514798,0.069039,0.092327
Travis,-1.361444,1.083480,0.758152,0.320153,1.094755


In [36]:
people.iloc[2:3, [1,2]]

,b,c
Wes,0.529197,0.670012


In [37]:
people.iloc[2:3, [1,2]] =  np.nan

In [38]:
people

,a,b,c,d,e
Joe,0.193067,-0.690679,-0.134004,0.718496,-0.360156
Steve,-0.031958,-0.407491,0.164615,-0.294408,-0.572945
Wes,-1.234819,NaN,NaN,-1.667063,-0.546139
Jim,0.918262,-2.794351,-1.514798,0.069039,0.092327
Travis,-1.361444,1.083480,0.758152,0.320153,1.094755


Now, suppose we have a group correspondence for the columns and want to sum together the columns by group:

In [39]:
mapping = {
    'a': 'red',
    'b': 'red', 
    'c': 'blue', 
    'd': 'blue', 
    'e': 'red',
    'f': 'orange'
}

Now, we could construct an array from this dict to pass to groupby, but instead we can just pass the dict

In [40]:
by_column = people.groupby(mapping, axis = 1)

In [41]:
by_column.sum()

,blue,red
Joe,0.584492,-0.857768
Steve,-0.129793,-1.012394
Wes,-1.667063,-1.780958
Jim,-1.445760,-1.783762
Travis,1.078305,0.816790


The same functionality holds for Sereis, which can be viewed as a fixed-size mapping:


In [42]:
map_series = pd.Series(mapping)

In [43]:
map_series

a       red
b       red
c      blue
d      blue
e       red
f    orange
dtype: object

In [44]:
type(map_series)

pandas.core.series.Series

In [45]:
people.groupby(map_series, axis = 1).count()

,blue,red
Joe,2,3
Steve,2,3
Wes,1,2
Jim,2,3
Travis,2,3


<h3>Grouping with Functions</h3>

Using Python functions is a more generic way of defining a group mapping compared with a dict or Series. Any function passed as a group key will be called once per index value, with the return values being used as the group names. More concretely, consider the example DataFrame from the previous section, which has people's first names as index values. Suppose we wanted to group by the length of the names; while we could compute an array of string lengths, it's simpler to just pass the len function:

In [46]:
people.groupby(len).sum()

,a,b,c,d,e
3,-0.123491,-3.485031,-1.648803,-0.879528,-0.813967
5,-0.031958,-0.407491,0.164615,-0.294408,-0.572945
6,-1.361444,1.083480,0.758152,0.320153,1.094755


Mixing functions with arrays, dicts, or Series is not a problem as everything gets converted to arrays internally:

In [47]:
key_list = ['one', 'one', 'one', 'two', 'two']

In [48]:
people.groupby([len, key_list]).min()

a         b         c         d         e
3 one -1.234819 -0.690679 -0.134004 -1.667063 -0.546139
  two  0.918262 -2.794351 -1.514798  0.069039  0.092327
5 one -0.031958 -0.407491  0.164615 -0.294408 -0.572945
6 two -1.361444  1.083480  0.758152  0.320153  1.094755

<h3>Grouping by Index Levels</h3>

A final convenience for hierarchially indexed datasets is the ability to aggregate using one of the levels of an axis index. 

In [49]:
columns = pd.MultiIndex.from_arrays([['US', 'US', 'US', 'JP', 'JP'],
                                    [1,3,5,1,3]],
                                   names = ['city', 'tenor'])

In [50]:
hier_df = pd.DataFrame(np.random.randn(4,5), columns = columns)

In [51]:
hier_df

city         US                            JP          
tenor         1         3         5         1         3
0      1.953126  1.215337  1.209559  0.031491 -0.985598
1     -0.806800  0.966906  0.053878 -0.140597  1.052035
2      0.488318  1.091479 -1.852972 -0.113795 -1.840805
3     -1.576409  0.350960 -0.253678 -1.326228  0.056655

To group by level, pass the level number or name using the <b>level</b> keyword:

In [52]:
hier_df.groupby(level='city', axis = 1).count()

city,JP,US
0,2,3
1,2,3
2,2,3
3,2,3


<h3>Data Aggregation</h3>

Aggregations refer to any data transformation that produces scalar values from arrays. The preceding examples have used several of them, including mean, count, min, and sum. We many wonder what is going on when we invoke mean() on a GroupBy object. Many common aggregations, such as those found in the table below have optimized implementations. However, we are not limited to only these set of methods

![alt Text](Images/DataAggregation/da_groupby.png)

We can use aggregations of our own devising and additionally call any method that is also defined on the grouped object. For example, we might recall that quantile computes sample quantiles of a Series or a DataFrame's columns.

While quantile is not explicitely implemented for GroupBy, it is a Series method and thus available for use. Internally, GroupBy efficiently slices up the Series, calls piece.quantile(0.9) for each piece, and then assembles those results together into the result object:

In [53]:
df

,key1,key2,data1,data2
0,a,one,-1.576350,0.460778
1,a,two,-0.541179,-0.873254
2,b,one,0.061983,-0.397402
3,b,two,-0.274277,0.645504
4,a,one,-1.427910,1.108724


In [54]:
grouped = df.groupby('key1')

In [55]:
grouped['data1'].quantile(0.9)

key1
a   -0.718525
b    0.028357
Name: data1, dtype: float64

To use our own aggregation functions, pass any function that aggregates an array to the aggregate or agg method:

In [56]:
def peak_to_peak(arr):
    return arr.max() - arr.min()

In [57]:
grouped.agg(peak_to_peak)

,data1,data2
key1,,
a,1.035171,1.981978
b,0.336260,1.042905


We can notice that some methods like <b>describe</b> also work, even though they are not aggregations, strictly speaking:

In [58]:
grouped.describe()

data1                                                              \
     count      mean       std       min       25%       50%       75%   
key1                                                                     
a      3.0 -1.181813  0.559748 -1.576350 -1.502130 -1.427910 -0.984545   
b      2.0 -0.106147  0.237771 -0.274277 -0.190212 -0.106147 -0.022082   

               data2                                                    \
           max count      mean       std       min       25%       50%   
key1                                                                     
a    -0.541179   3.0  0.232082  1.010587 -0.873254 -0.206238  0.460778   
b     0.061983   2.0  0.124051  0.737446 -0.397402 -0.136675  0.124051   

                          
           75%       max  
key1                      
a     0.784751  1.108724  
b     0.384777  0.645504

<h3>Column-Wise and Multiple Function Application</h3>

In [59]:
tips = pd.read_csv('pydata-book-2nd-edition/examples/tips.csv')

In [60]:
tips['tip_pct'] = tips['tip'] / tips['total_bill']

In [61]:
tips[:6]

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808
5,25.29,4.71,No,Sun,Dinner,4,0.186240


As we have already seen, aggregating a Series or all of the columns of a DataFrame is a matter of using aggregate with the desired function or calling a method like mean or std. However, we may want to aggregate using a different function depending on the column, or multiple functions at once.

In [62]:
grouped = tips.groupby(['day', 'smoker'])

In [63]:
grouped_pct = grouped['tip_pct']

In [64]:
grouped_pct.agg('mean')

day   smoker
Fri   No        0.151650
      Yes       0.174783
Sat   No        0.158048
      Yes       0.147906
Sun   No        0.160113
      Yes       0.187250
Thur  No        0.160298
      Yes       0.163863
Name: tip_pct, dtype: float64

If we pass a list of functions or function names instead, we get back a DataFrame with column names taken from the functions:

In [65]:
grouped_pct.agg(['mean', 'std', peak_to_peak])

mean       std  peak_to_peak
day  smoker                                  
Fri  No      0.151650  0.028123      0.067349
     Yes     0.174783  0.051293      0.159925
Sat  No      0.158048  0.039767      0.235193
     Yes     0.147906  0.061375      0.290095
Sun  No      0.160113  0.042347      0.193226
     Yes     0.187250  0.154134      0.644685
Thur No      0.160298  0.038774      0.193350
     Yes     0.163863  0.039389      0.151240

Here, we passed a list of aggregation functions to agg to evaluate independently on the data groups.

We don't need to accept the names that GroupBy gives to the columns; notably, <b>lambda</b> functions have the name '<lambda>' which makes them hard to identify. Thus, if we pass a list of (name, function) tuples, the first element of each tuple will be used as the DataFrame column names

In [66]:
grouped_pct.agg([('foo', 'mean'), ('bar', np.std)])

foo       bar
day  smoker                    
Fri  No      0.151650  0.028123
     Yes     0.174783  0.051293
Sat  No      0.158048  0.039767
     Yes     0.147906  0.061375
Sun  No      0.160113  0.042347
     Yes     0.187250  0.154134
Thur No      0.160298  0.038774
     Yes     0.163863  0.039389

With a DataFrame, we have more options, as we can specify a list of functions to apply to all of the columns or different functions per column. To start, suppose we wanted to compute the same three statistics for the tip_pct and total_bill columns:

In [67]:
functions = ['count', 'mean', 'max']

In [68]:
result = grouped['tip_pct', 'total_bill'].agg(functions)

C:\Users\Predator\Anaconda3\envs\data_analysis\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """Entry point for launching an IPython kernel.


In [69]:
result

tip_pct                     total_bill                  
              count      mean       max      count       mean    max
day  smoker                                                         
Fri  No           4  0.151650  0.187735          4  18.420000  22.75
     Yes         15  0.174783  0.263480         15  16.813333  40.17
Sat  No          45  0.158048  0.291990         45  19.661778  48.33
     Yes         42  0.147906  0.325733         42  21.276667  50.81
Sun  No          57  0.160113  0.252672         57  20.506667  48.17
     Yes         19  0.187250  0.710345         19  24.120000  45.35
Thur No          45  0.160298  0.266312         45  17.113111  41.19
     Yes         17  0.163863  0.241255         17  19.190588  43.11

As we can see, the resulting DataFrame has hierarchial columns, the same as we would get aggregating each column separately and using <b>concat</b> to glue the results together using the column names as the keys argument:

In [70]:
result['tip_pct']

count      mean       max
day  smoker                           
Fri  No          4  0.151650  0.187735
     Yes        15  0.174783  0.263480
Sat  No         45  0.158048  0.291990
     Yes        42  0.147906  0.325733
Sun  No         57  0.160113  0.252672
     Yes        19  0.187250  0.710345
Thur No         45  0.160298  0.266312
     Yes        17  0.163863  0.241255

As before, a list of tuples with custom names can be passed:


In [71]:
ftuples = [('Durchschnitt', 'mean'), ('Abweichung', np.var)]

In [72]:
grouped['tip_pct', 'total_bill'].agg(ftuples)

C:\Users\Predator\Anaconda3\envs\data_analysis\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """Entry point for launching an IPython kernel.


tip_pct              total_bill            
            Durchschnitt Abweichung Durchschnitt  Abweichung
day  smoker                                                 
Fri  No         0.151650   0.000791    18.420000   25.596333
     Yes        0.174783   0.002631    16.813333   82.562438
Sat  No         0.158048   0.001581    19.661778   79.908965
     Yes        0.147906   0.003767    21.276667  101.387535
Sun  No         0.160113   0.001793    20.506667   66.099980
     Yes        0.187250   0.023757    24.120000  109.046044
Thur No         0.160298   0.001503    17.113111   59.625081
     Yes        0.163863   0.001551    19.190588   69.808518

Now, suppose we wanted to apply potentially different functions to one or more of the columns. To do this, pass a dict to agg that contains a mapping of column names to any of the function specifications listed so far:

In [73]:
grouped.agg({'tip': np.max, 'size': 'sum'})

tip  size
day  smoker             
Fri  No       3.50     9
     Yes      4.73    31
Sat  No       9.00   115
     Yes     10.00   104
Sun  No       6.00   167
     Yes      6.50    49
Thur No       6.70   112
     Yes      5.00    40

In [74]:
grouped.agg({'tip_pct': ['min', 'max', 'mean', 'std'],
            'size': 'sum'})

tip_pct                               size
                  min       max      mean       std  sum
day  smoker                                             
Fri  No      0.120385  0.187735  0.151650  0.028123    9
     Yes     0.103555  0.263480  0.174783  0.051293   31
Sat  No      0.056797  0.291990  0.158048  0.039767  115
     Yes     0.035638  0.325733  0.147906  0.061375  104
Sun  No      0.059447  0.252672  0.160113  0.042347  167
     Yes     0.065660  0.710345  0.187250  0.154134   49
Thur No      0.072961  0.266312  0.160298  0.038774  112
     Yes     0.090014  0.241255  0.163863  0.039389   40

<h3>Returning Aggregated Data Without Row Indexes</h3>

In all of the exmples up until now, the aggregated data comes back with an index, potentially hierarchial, composed from the unique group key combinations. Since this isn't always desirable, we can disable this behavior in most cases by passing as_index = False to groupby:

In [75]:
tips.groupby(['day', 'smoker'], as_index=False).mean()

,day,smoker,total_bill,tip,size,tip_pct
0,Fri,No,18.420000,2.812500,2.250000,0.151650
1,Fri,Yes,16.813333,2.714000,2.066667,0.174783
2,Sat,No,19.661778,3.102889,2.555556,0.158048
3,Sat,Yes,21.276667,2.875476,2.476190,0.147906
4,Sun,No,20.506667,3.167895,2.929825,0.160113
5,Sun,Yes,24.120000,3.516842,2.578947,0.187250
6,Thur,No,17.113111,2.673778,2.488889,0.160298
7,Thur,Yes,19.190588,3.030000,2.352941,0.163863


<h3>Apply: General split-apply-combine</h3>

Returning to the tipping dataset from before, suppose we wanted to select the top five tip_pct values by group. First, write a function that selects the rows with the largest values in a particular column:

In [76]:
def top(df, n=5, column = 'tip_pct'):
    return df.sort_values(by=column)[-n:]

In [77]:
top(tips, n = 6)

,total_bill,tip,smoker,day,time,size,tip_pct
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
232,11.61,3.39,No,Sat,Dinner,2,0.291990
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345


Now, if we group by smoker, say, and call apply with this function, we get the following:

In [78]:
tips.groupby('smoker').apply(top)

total_bill   tip smoker   day    time  size   tip_pct
smoker                                                           
No     88        24.71  5.85     No  Thur   Lunch     2  0.236746
       185       20.69  5.00     No   Sun  Dinner     5  0.241663
       51        10.29  2.60     No   Sun  Dinner     2  0.252672
       149        7.51  2.00     No  Thur   Lunch     2  0.266312
       232       11.61  3.39     No   Sat  Dinner     2  0.291990
Yes    109       14.31  4.00    Yes   Sat  Dinner     2  0.279525
       183       23.17  6.50    Yes   Sun  Dinner     4  0.280535
       67         3.07  1.00    Yes   Sat  Dinner     1  0.325733
       178        9.60  4.00    Yes   Sun  Dinner     2  0.416667
       172        7.25  5.15    Yes   Sun  Dinner     2  0.710345

Here, thetop function is called on each row group from the DataFrame, and then the results are glued together using pandas.concat, labeling the piecese with the group names. The result therefore has a hierarchial index whose inner level contains index values from the original DataFrame.

If we pass a function to apply that takes other arguements or keywords, we can pass these after the function:

In [79]:
tips

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808
...,...,...,...,...,...,...,...
239,29.03,5.92,No,Sat,Dinner,3,0.203927
240,27.18,2.00,Yes,Sat,Dinner,2,0.073584
241,22.67,2.00,Yes,Sat,Dinner,2,0.088222
242,17.82,1.75,No,Sat,Dinner,2,0.098204


In [80]:
tips.groupby(['smoker', 'day']).apply(top, n= 1, column = 'total_bill')

total_bill    tip smoker   day    time  size   tip_pct
smoker day                                                             
No     Fri  94        22.75   3.25     No   Fri  Dinner     2  0.142857
       Sat  212       48.33   9.00     No   Sat  Dinner     4  0.186220
       Sun  156       48.17   5.00     No   Sun  Dinner     6  0.103799
       Thur 142       41.19   5.00     No  Thur   Lunch     5  0.121389
Yes    Fri  95        40.17   4.73    Yes   Fri  Dinner     4  0.117750
       Sat  170       50.81  10.00    Yes   Sat  Dinner     3  0.196812
       Sun  182       45.35   3.50    Yes   Sun  Dinner     3  0.077178
       Thur 197       43.11   5.00    Yes  Thur   Lunch     4  0.115982

We can call <b>describe</b> on a GroupBy object:

In [81]:
result = tips.groupby('smoker')['tip_pct'].describe()

In [82]:
result

,count,mean,std,min,25%,50%,75%,max
smoker,,,,,,,,
No,151.0,0.159328,0.039910,0.056797,0.136906,0.155625,0.185014,0.291990
Yes,93.0,0.163196,0.085119,0.035638,0.106771,0.153846,0.195059,0.710345


In [83]:
result.unstack('smoker')

       smoker
count  No        151.000000
       Yes        93.000000
mean   No          0.159328
       Yes         0.163196
std    No          0.039910
       Yes         0.085119
min    No          0.056797
       Yes         0.035638
25%    No          0.136906
       Yes         0.106771
50%    No          0.155625
       Yes         0.153846
75%    No          0.185014
       Yes         0.195059
max    No          0.291990
       Yes         0.710345
dtype: float64

Inside GroupBy, when we invoke a method like desfcribe, it is actually just a short-cut for:
<pre>
    f = lambda x: x.describe()
    grouped.apply(f)
</pre>

<h3>Suppressing the Group Keys</h3>

In the preceding examples, we can see that the resulting object has a hierarchial index formed from the group keys along with the indexes of each piece of the original object. 
We can disable this by passing group_keys = False to groupby:

In [84]:
tips.groupby('smoker').apply(top)

total_bill   tip smoker   day    time  size   tip_pct
smoker                                                           
No     88        24.71  5.85     No  Thur   Lunch     2  0.236746
       185       20.69  5.00     No   Sun  Dinner     5  0.241663
       51        10.29  2.60     No   Sun  Dinner     2  0.252672
       149        7.51  2.00     No  Thur   Lunch     2  0.266312
       232       11.61  3.39     No   Sat  Dinner     2  0.291990
Yes    109       14.31  4.00    Yes   Sat  Dinner     2  0.279525
       183       23.17  6.50    Yes   Sun  Dinner     4  0.280535
       67         3.07  1.00    Yes   Sat  Dinner     1  0.325733
       178        9.60  4.00    Yes   Sun  Dinner     2  0.416667
       172        7.25  5.15    Yes   Sun  Dinner     2  0.710345

In [85]:
tips.groupby('smoker', as_index=False).apply(top)

total_bill   tip smoker   day    time  size   tip_pct
0 88        24.71  5.85     No  Thur   Lunch     2  0.236746
  185       20.69  5.00     No   Sun  Dinner     5  0.241663
  51        10.29  2.60     No   Sun  Dinner     2  0.252672
  149        7.51  2.00     No  Thur   Lunch     2  0.266312
  232       11.61  3.39     No   Sat  Dinner     2  0.291990
1 109       14.31  4.00    Yes   Sat  Dinner     2  0.279525
  183       23.17  6.50    Yes   Sun  Dinner     4  0.280535
  67         3.07  1.00    Yes   Sat  Dinner     1  0.325733
  178        9.60  4.00    Yes   Sun  Dinner     2  0.416667
  172        7.25  5.15    Yes   Sun  Dinner     2  0.710345

In [86]:
tips.groupby('smoker',group_keys=False).apply(top)

,total_bill,tip,smoker,day,time,size,tip_pct
88,24.71,5.85,No,Thur,Lunch,2,0.236746
185,20.69,5.00,No,Sun,Dinner,5,0.241663
51,10.29,2.60,No,Sun,Dinner,2,0.252672
149,7.51,2.00,No,Thur,Lunch,2,0.266312
232,11.61,3.39,No,Sat,Dinner,2,0.291990
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345


<h3>Quantile and Bucket Analysis</h3>

As we can recall, pandas has some tools, in particualr <b>cut</b> and <b>qcut</b> for slicing data up into the buckets with bins of our choosing or by sampe quantiles. Combining these functions with groupby makes it convenient to perfrom bucket or quantile analysis on a dataset. Consider a simple random dataset and an equal-length bucket categorization using cut:

In [87]:
frame = pd.DataFrame({
    'data1': np.random.randn(1000),
    'data2': np.random.randn(1000)
})

In [88]:
frame

,data1,data2
0,-0.490100,-0.849731
1,0.025210,-2.324746
2,0.361027,-1.779333
3,-0.071158,0.375730
4,1.391615,-0.239182
...,...,...
995,0.588300,-0.067371
996,-0.907226,-0.864938
997,-0.416665,-0.116574
998,-1.643329,-0.548242


In [89]:
quartiles = pd.cut(frame.data1,4)

In [90]:
quartiles[:10]

0    (-1.564, 0.196]
1    (-1.564, 0.196]
2     (0.196, 1.956]
3    (-1.564, 0.196]
4     (0.196, 1.956]
5     (0.196, 1.956]
6     (1.956, 3.716]
7     (0.196, 1.956]
8     (0.196, 1.956]
9    (-1.564, 0.196]
Name: data1, dtype: category
Categories (4, interval[float64]): [(-3.331, -1.564] < (-1.564, 0.196] < (0.196, 1.956] < (1.956, 3.716]]

The Categoraical object returned by cut can be passed directly into grouopby. So we could compute a set of statistics for the data2 column like so:

In [91]:
def get_stats(group):
    return {
        'min': group.min(),
        'max': group.max(),
        'count': group.count(),
        'mean': group.mean()
    }

In [92]:
frame

,data1,data2
0,-0.490100,-0.849731
1,0.025210,-2.324746
2,0.361027,-1.779333
3,-0.071158,0.375730
4,1.391615,-0.239182
...,...,...
995,0.588300,-0.067371
996,-0.907226,-0.864938
997,-0.416665,-0.116574
998,-1.643329,-0.548242


In [93]:
grouped = frame.data2.groupby(quartiles)

In [94]:
grouped.apply(get_stats).unstack()

,min,max,count,mean
data1,,,,
"(-3.331, -1.564]",-1.674603,2.193914,59.0,0.207669
"(-1.564, 0.196]",-3.067058,3.236032,512.0,-0.026369
"(0.196, 1.956]",-3.337214,3.226647,403.0,0.086239
"(1.956, 3.716]",-1.687064,1.919387,26.0,0.000517


These were equal-length buckets; to compute equal-size buckets based on sample quantiles, use qcut. we will pass labels = False, to just get quantile numberes:

In [95]:
frame

,data1,data2
0,-0.490100,-0.849731
1,0.025210,-2.324746
2,0.361027,-1.779333
3,-0.071158,0.375730
4,1.391615,-0.239182
...,...,...
995,0.588300,-0.067371
996,-0.907226,-0.864938
997,-0.416665,-0.116574
998,-1.643329,-0.548242


In [96]:
grouping = pd.qcut(frame.data1, 10,labels = False)

In [97]:
grouped = frame.data2.groupby(grouping)

In [98]:
grouped.apply(get_stats).unstack()

,min,max,count,mean
data1,,,,
0,-3.067058,2.193914,100.0,0.144479
1,-2.419945,2.220437,100.0,-0.059289
2,-2.720099,2.662568,100.0,-0.087115
3,-2.582265,3.236032,100.0,0.046100
4,-2.295814,2.159522,100.0,0.049939
5,-2.324746,2.283904,100.0,-0.073876
6,-2.821112,3.226647,100.0,0.183261
7,-2.201880,2.296975,100.0,0.139990
8,-2.701189,2.996810,100.0,0.029632


<h3>Filling Missing Values with Group-Specific Values</h3>

When cleaning up missing data, in some cases we will replace data observations using dropna, but in others we may want to fill in the null values using a fixed value or some value derived from the data. 
<b>fillna</b> is the right tool to use; for example here we will fill in NA values with the mean:

In [99]:
s = pd.Series(np.random.randn(6))

In [100]:
s

0   -1.156850
1    0.178484
2   -0.175079
3   -1.167292
4    1.436820
5    0.344797
dtype: float64

In [101]:
s[::2] = np.nan

In [102]:
s

0         NaN
1    0.178484
2         NaN
3   -1.167292
4         NaN
5    0.344797
dtype: float64

In [103]:
s.fillna(s.mean())

0   -0.214670
1    0.178484
2   -0.214670
3   -1.167292
4   -0.214670
5    0.344797
dtype: float64

Suppose we need to fill value to vary by group. One way to do this is to group the data and use apply with a function that calls fillna on each data chunk. Here is some sample data on US states divided into eastern and western regions:

In [104]:
states = ['Ohio', 'New York', 'Vermont', 'Florida', 'Oregon', 'Nevada',
         'California', 'Idaho']

In [105]:
group_key = ['East']*4 + ['West']*4

Note that the syntax ['East']*4 + ['West']*4 produces a list containing four copies of teh elements in ['East']. Adding lists together concatenates them.

In [106]:
data = pd.Series(np.random.randn(8), index = states)

In [107]:
data

Ohio          0.918509
New York     -0.092627
Vermont      -0.227199
Florida      -0.231023
Oregon       -0.046749
Nevada        0.409201
California    0.810885
Idaho        -0.325054
dtype: float64

In [108]:
data[['Vermont', 'Nevada', 'Idaho']] = np.nan

In [109]:
data

Ohio          0.918509
New York     -0.092627
Vermont            NaN
Florida      -0.231023
Oregon       -0.046749
Nevada             NaN
California    0.810885
Idaho              NaN
dtype: float64

In [110]:
data.groupby(group_key).mean()

East    0.198286
West    0.382068
dtype: float64

We can fill the NA values using the group means like so:

In [111]:
fill_mean = lambda g:g.fillna(g.mean())

In [112]:
data.groupby(group_key).apply(fill_mean)

Ohio          0.918509
New York     -0.092627
Vermont       0.198286
Florida      -0.231023
Oregon       -0.046749
Nevada        0.382068
California    0.810885
Idaho         0.382068
dtype: float64

In another case, we might have predefined fill values in our code that vary by gorup. Since the groups have a name attribute set internally, we can use that:

In [113]:
fill_values = {
    'East': 0.5,
    'West': -1
}

In [114]:
fill_func = lambda g: g.fillna(fill_values[g.name])

In [115]:
data

Ohio          0.918509
New York     -0.092627
Vermont            NaN
Florida      -0.231023
Oregon       -0.046749
Nevada             NaN
California    0.810885
Idaho              NaN
dtype: float64

In [116]:
data.groupby(group_key).apply(fill_func)

Ohio          0.918509
New York     -0.092627
Vermont       0.500000
Florida      -0.231023
Oregon       -0.046749
Nevada       -1.000000
California    0.810885
Idaho        -1.000000
dtype: float64

<h3>Random Sampling and Permutation</h3>

Suppose we wanted to draw a random sample from a large dataset for Monte Carlo simulation purposees or some other application. There are a number of ways to perform the 'draws'; here we use the <b>sample</b> method for Series

In [117]:
suits = ['H', 'S', 'C', 'D']

In [118]:
card_val = (list(range(1,11))+ [10]*3) * 4

In [119]:
base_names = ['A'] + list(range(2,11)) + ['J', 'K', 'Q']

In [120]:
cards = []

In [121]:
for suit in ['H','S', 'C', 'D']:
    cards.extend(str(num) + suit for num in base_names)

In [122]:
deck = pd.Series(card_val, index=cards)

In [123]:
deck[:13]

AH      1
2H      2
3H      3
4H      4
5H      5
6H      6
7H      7
8H      8
9H      9
10H    10
JH     10
KH     10
QH     10
dtype: int64

So now we have a Series of length 52 whose index contains card names and values are the ones used in Blackjack and other games

In [124]:
deck[:13]

AH      1
2H      2
3H      3
4H      4
5H      5
6H      6
7H      7
8H      8
9H      9
10H    10
JH     10
KH     10
QH     10
dtype: int64

Now, based on what I said before, drawing a hand of five cards from the deck could be written as:

In [125]:
def draw(deck, n =5):
    return deck.sample(n)

In [126]:
draw(deck)

4D    4
4S    4
9H    9
2C    2
2D    2
dtype: int64

Suppose we wanted two random cards from each suit. Because the suit is the last character of each card name, we can group based on this and use apply:

In [127]:
get_suit = lambda card: card[-1]

In [128]:
deck.groupby(get_suit).apply(draw, n=2)

C  QC     10
   JC     10
D  10D    10
   JD     10
H  3H      3
   10H    10
S  QS     10
   4S      4
dtype: int64

Alternatively we could write:

In [129]:
deck.groupby(get_suit, group_keys=False).apply(draw, n = 2)

10C    10
8C      8
AD      1
2D      2
5H      5
9H      9
3S      3
10S    10
dtype: int64

<h3>Examples: Group Weighted Average and Correlation</h3>

Under the split-apply-combine paradigm of groupby, operations between columns in a DataFrame or two Series, such as a group weighted average, are possible. As an example, take this dataset containing group keys, values and some weights:

In [130]:
df = pd.DataFrame({
    'category': ['a', 'a', 'a', 'a', 'b', 'b','b','b'],
    'data': np.random.randn(8),
    'weights': np.random.rand(8)
})

In [131]:
df

,category,data,weights
0,a,1.035811,0.794725
1,a,-1.772162,0.779289
2,a,0.081540,0.963068
3,a,0.627302,0.046854
4,b,1.327580,0.793533
5,b,-1.911247,0.920624
6,b,-1.847701,0.368760
7,b,0.065364,0.296450


The group weighted averate by category would then be:

In [132]:
grouped = df.groupby('category')

In [133]:
get_wavg = lambda g: np.average(g['data'], weights = g['weights'])

In [134]:
grouped.apply(get_wavg)

category
a   -0.174122
b   -0.574960
dtype: float64

As another example, cosider a financial dataset originally obtained form Yahoo! Finance containing end-of-day prices for a few stocks and teh S&P 500 index

In [135]:
close_px = pd.read_csv('pydata-book-2nd-edition/examples/stock_px_2.csv',
                      parse_dates = True, index_col = 0)

In [136]:
close_px.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2214 entries, 2003-01-02 to 2011-10-14
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   AAPL    2214 non-null   float64
 1   MSFT    2214 non-null   float64
 2   XOM     2214 non-null   float64
 3   SPX     2214 non-null   float64
dtypes: float64(4)
memory usage: 86.5 KB


In [137]:
close_px[-4:]

,AAPL,MSFT,XOM,SPX
2011-10-11,400.29,27.00,76.27,1195.54
2011-10-12,402.19,26.96,77.16,1207.25
2011-10-13,408.43,27.18,76.37,1203.66
2011-10-14,422.00,27.27,78.11,1224.58


One task of interest might be to compute a DataFrame consisting of the yearly correlations of daily returns with SPX. As one way to do this, we first create  function that computes the pairwise correlation of each column with the 'SPX' column:

In [138]:
spx_corr = lambda x:x.corrwith(x['SPX'])

Next, we compute percent change on close_px using pct_change:

In [139]:
rets = close_px.pct_change().dropna()

Lastly, we group these percent changes by year, which can be extracted from each row label with a one-line function that returns the year attribute of each datetime label:

In [140]:
get_year = lambda x: x.year

In [141]:
by_year = rets.groupby(get_year)

In [142]:
by_year.apply(spx_corr)

,AAPL,MSFT,XOM,SPX
2003,0.541124,0.745174,0.661265,1.0
2004,0.374283,0.588531,0.557742,1.0
2005,0.467540,0.562374,0.631010,1.0
2006,0.428267,0.406126,0.518514,1.0
2007,0.508118,0.658770,0.786264,1.0
2008,0.681434,0.804626,0.828303,1.0
2009,0.707103,0.654902,0.797921,1.0
2010,0.710105,0.730118,0.839057,1.0
2011,0.691931,0.800996,0.859975,1.0


We could also compute inter-column correlations. Here, we compute the annual correlation between Apple and Microsoft:

In [143]:
by_year.apply(lambda g: g['AAPL'].corr(g['MSFT']))

2003    0.480868
2004    0.259024
2005    0.300093
2006    0.161735
2007    0.417738
2008    0.611901
2009    0.432738
2010    0.571946
2011    0.581987
dtype: float64

<h3>Group Wise Linear Regression </h3>

We cn use groupby to perform more complex group-wise statistical analysis, as long as the function returns a pandas obect or scalar value. For example, I can define the following regress function, which executes an ordinary least squares (OLS) regression on each chunk of data:

In [144]:
import statsmodels.api as sm

In [145]:
def regress(data, yvar, xvars):
    Y = data[yvar]
    X = data[xvars]
    X['intercept'] = 1
    result = sm.OLS(Y,X).fit()
    return result.params

Now, to run a yearly lineary regression of AAPL on SPX returns, execute:

In [146]:
by_year.apply(regress, 'AAPL', ['SPX'])

,SPX,intercept
2003,1.195406,0.000710
2004,1.363463,0.004201
2005,1.766415,0.003246
2006,1.645496,0.000080
2007,1.198761,0.003438
2008,0.968016,-0.001110
2009,0.879103,0.002954
2010,1.052608,0.001261
2011,0.806605,0.001514


<h3>Pivot Tables and Cross Tabulation</h3>

A <b>pivot table</b> is a data summarization tool frequently found in spreadhseet programs and other data analysis software. It aggregates a table of data by one or mroe keys, arranging the data in a rectangle with some of the goru pkeys along the rows and some along the columns. Pivot tables in Python with pandas are made possible through the <b>groupby</b> facility described in this chapter combined with reshape operations utilizing hierarchial indexing. DataFrame has a pivot_table method, and there is also a top-level pandas.pivot_table function. In addition to providing a convenience interface to gropuby, pivot_table can add partial totals, also known as margins.

Returning to the tipping dataset, suppose we wanted to compute a table of group means arrange by dat and smoker on the rows:

In [147]:
tips

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808
...,...,...,...,...,...,...,...
239,29.03,5.92,No,Sat,Dinner,3,0.203927
240,27.18,2.00,Yes,Sat,Dinner,2,0.073584
241,22.67,2.00,Yes,Sat,Dinner,2,0.088222
242,17.82,1.75,No,Sat,Dinner,2,0.098204


In [148]:
tips.pivot_table(index = ['day', 'smoker'])

size       tip   tip_pct  total_bill
day  smoker                                          
Fri  No      2.250000  2.812500  0.151650   18.420000
     Yes     2.066667  2.714000  0.174783   16.813333
Sat  No      2.555556  3.102889  0.158048   19.661778
     Yes     2.476190  2.875476  0.147906   21.276667
Sun  No      2.929825  3.167895  0.160113   20.506667
     Yes     2.578947  3.516842  0.187250   24.120000
Thur No      2.488889  2.673778  0.160298   17.113111
     Yes     2.352941  3.030000  0.163863   19.190588

This could have been produced with groupby directly. Now, suppose we want to aggregate only tip_pct and size, and additionallly group by time. I'll put smoker in the table columns and day in the rows:

In [149]:
tips.pivot_table(['tip_pct', 'size'],
                index = ['time', 'day'],
                columns = 'smoker')

size             tip_pct          
smoker             No       Yes        No       Yes
time   day                                         
Dinner Fri   2.000000  2.222222  0.139622  0.165347
       Sat   2.555556  2.476190  0.158048  0.147906
       Sun   2.929825  2.578947  0.160113  0.187250
       Thur  2.000000       NaN  0.159744       NaN
Lunch  Fri   3.000000  1.833333  0.187735  0.188937
       Thur  2.500000  2.352941  0.160311  0.163863

We could augment this table to include partial totals by passing margins = True. This has the effect of adding All row and column labels, with corresponding values being the group statistics for all the data  within a single tier:

In [150]:
tips.pivot_table(['tip_pct', 'size'],
                index = ['time', 'day'],
                columns = 'smoker',
                margins = True)

size                       tip_pct                    
smoker             No       Yes       All        No       Yes       All
time   day                                                             
Dinner Fri   2.000000  2.222222  2.166667  0.139622  0.165347  0.158916
       Sat   2.555556  2.476190  2.517241  0.158048  0.147906  0.153152
       Sun   2.929825  2.578947  2.842105  0.160113  0.187250  0.166897
       Thur  2.000000       NaN  2.000000  0.159744       NaN  0.159744
Lunch  Fri   3.000000  1.833333  2.000000  0.187735  0.188937  0.188765
       Thur  2.500000  2.352941  2.459016  0.160311  0.163863  0.161301
All          2.668874  2.408602  2.569672  0.159328  0.163196  0.160803

Here, the ALL values are means without taking into account smoker versus non-smoker or any of the two levels of grouping on the rows

To use a different aggregation function, pass it to aggfunc. For example, 'count' or len will give you a cross-tabulation(count or frequency) of group sizes:

In [151]:
tips.pivot_table('tip_pct', index = ['time', 'smoker'], 
                columns = 'day', aggfunc = len, margins = True)

day             Fri   Sat   Sun  Thur    All
time   smoker                               
Dinner No       3.0  45.0  57.0   1.0  106.0
       Yes      9.0  42.0  19.0   NaN   70.0
Lunch  No       1.0   NaN   NaN  44.0   45.0
       Yes      6.0   NaN   NaN  17.0   23.0
All            19.0  87.0  76.0  62.0  244.0

If some combinations are empty, we may wish to pass a fill_value:

In [152]:
tips.pivot_table('tip_pct', index = ['time', 'size', 'smoker'],
                columns = 'day', aggfunc = 'mean', fill_value = 0)

day                      Fri       Sat       Sun      Thur
time   size smoker                                        
Dinner 1    No      0.000000  0.137931  0.000000  0.000000
            Yes     0.000000  0.325733  0.000000  0.000000
       2    No      0.139622  0.162705  0.168859  0.159744
            Yes     0.171297  0.148668  0.207893  0.000000
       3    No      0.000000  0.154661  0.152663  0.000000
            Yes     0.000000  0.144995  0.152660  0.000000
       4    No      0.000000  0.150096  0.148143  0.000000
            Yes     0.117750  0.124515  0.193370  0.000000
       5    No      0.000000  0.000000  0.206928  0.000000
            Yes     0.000000  0.106572  0.065660  0.000000
       6    No      0.000000  0.000000  0.103799  0.000000
Lunch  1    No      0.000000  0.000000  0.000000  0.181728
            Yes     0.223776  0.000000  0.000000  0.000000
       2    No      0.000000  0.000000  0.000000  0.166005
            Yes     0.181969  0.000000  0.000000  0.158843
       3    No      0.187735  0.000000  0.000000  0.084246
            Yes     0.000000  0.000000  0.000000  0.204952
       4    No      0.000000  0.000000  0.000000  0.138919
            Yes     0.000000  0.000000  0.000000  0.155410
       5    No      0.000000  0.000000  0.000000  0.121389
       6    No      0.000000  0.000000  0.000000  0.173706

![alt Text](Images/DataAggregation/da_pivottable.png)

<h3>Cross-Tabulations: Crosstab</h3>

A cross-tabulation is a special case of a pivot table that computes group frequencies. Here is an example:

In [153]:
data = pd.DataFrame({
    'sample' : np.arange(1,11),
    'nationality': ['usa', 'japan', 'usa', 'japan','japan', 'japan',
                   'usa', 'usa', 'japan', 'usa'],
    'handedness': ['right', 'left', 'right', 'right', 'left', 
                  'right', 'right', 'left', 'right', 'right']
})

In [154]:
data

,sample,nationality,handedness
0,1,usa,right
1,2,japan,left
2,3,usa,right
3,4,japan,right
4,5,japan,left
5,6,japan,right
6,7,usa,right
7,8,usa,left
8,9,japan,right
9,10,usa,right


As part of some survey analysis, we migth want to summarize this data by nationality and handedness. We could use pivot_table to do this, but the <b>pandas.crosstab</b> function can be more convenient:

In [155]:
pd.crosstab(data.nationality, data.handedness)

handedness,left,right
nationality,,
japan,2,3
usa,1,4


In [156]:
pd.crosstab(data.nationality, data.handedness, margins = True)

handedness,left,right,All
nationality,,,
japan,2,3,5
usa,1,4,5
All,3,7,10


The first two arguments to crosstab can each either be an array or Series or a list of arrays. As in the tips data:


In [157]:
pd.crosstab([tips.time, tips.day], tips.smoker, margins = True)

smoker        No  Yes  All
time   day                
Dinner Fri     3    9   12
       Sat    45   42   87
       Sun    57   19   76
       Thur    1    0    1
Lunch  Fri     1    6    7
       Thur   44   17   61
All          151   93  244